In [4]:
import nltk 
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [103]:
from collections import defaultdict
import heapq

class TrieNode:
    def __init__(self):
        self.children = defaultdict(TrieNode)
        self.is_end_of_word = False
        self.frequency =0
        self.word =None

class Trie:
    def __init__(self):
        self.root =TrieNode()

    def insert(self, word, frequency =1):
        node = self.root
        for char in word:
            node =  node.children[char]
        node.is_end_of_word =True
        node.frequency +=frequency
        node.word =word

    def autocomplete(self, prefix, top_n =3):
        node =self.root
        for char in prefix:
            if char not in node.children:
                return []
            node = node.children[char]


        result_heap = []

        def dfs(current_node):
            if current_node.is_end_of_word:
                heapq.heappush(result_heap, (-current_node.frequency, current_node.word))
            for child in current_node.children.values():
                dfs(child)


        dfs(node)

        top_suggestions =[]
        for _ in range(min(top_n, len(result_heap))):
            freq, word = heapq.heappop(result_heap)
            top_suggestions.append((word, -freq))
    
        return top_suggestions


    def increase_frequency(self, word, amount =1): #for auto-learning frequency
        node = self.root
        for char in word:
            if char not in node.children:
                print(f"Word {word} not found in Trie")
                return False
            node = node.children[char]

        if node.is_end_of_word:
            node.frequency+= amount
            print(f"Frequency of {word} updated to {node.frequency}")
            return True
        else:
            print(f"{word} is not a complete word in Trie.")
            return False

### 1. Levenshtein Eidt Distance Function

In [112]:
def edit_distance(word1, word2):
    m, n =len(word1), len(word2)
    dp = [[0]*(n+1) for _ in range(m+1)]

    for i in range(m+1):
        dp[i][0] =i
    for j in range(n+1):
        dp[0][j] =j

    for i in range(1, m+1):
        for j in range(1, n+1):
            if word1[i-1] == word2[j-1]:
                dp[i][j] =dp[i-1][j-1]
            else:
                dp[i][j] =1+ min(dp[i-1][j], #delete
                                 dp[i][j-1], #insert
                                 dp[i-1][j-1])  #replace

    return dp[m][n]


####### for Spelling Corrector using Levenshtein
def correct_prefix(trie, typo_prefix, max_distance =2):
    candidates = []
    all_words = collect_all_words(trie.root)

    for word,freq in all_words:
        if len(word) >= len(typo_prefix):
            w_prefix =word[:len(typo_prefix)]
            distance = edit_distance(typo_prefix, w_prefix)
            if distance <= max_distance:
                candidates.append((distance, -freq, w_prefix))


    if not candidates:
        return None

    # Sort by lowes distance, highest frequency
    candidates.sort
    return candidates[0][2] #return corrected prefix


def smart_autocomplete(trie, raw_prefix, top_n=5):
    #Try exact match first
    results = trie.autocomplete(raw_prefix, top_n)
    if results:
        return results

    #Otherwise correct the prefix
    corrected =correct_prefix(trie, raw_prefix)
    if corrected:
        print(f"Prefix {raw_prefix} corrected to {corrected}")
        return trie.autocomplete(corrected, top_n)
    else:
        print(f"No correction found for {raw_prefix}")
        return []

### 2. Helper to Collect All Words from Trie

In [115]:
# This helper to the notebook to get all valid words + frequencies form the Trie:


def collect_all_words(node, words =None):
    if words is None:
        words =[]

    if node.is_end_of_word:
        words.append((node.word , node.frequency))


    for child in node.children.values():  # ✅ correct method
        collect_all_words(child, words)

    return words

### 3. Fuzzy Autocomplete Function

In [118]:
def fuzzy_autocomplete(trie, typo_prefix, max_distance =1 , top_n=5):
    all_words = collect_all_words(trie.root)
    close_matches = []

    for word, freq in all_words:
        prefix_part =word[:len(typo_prefix)]
        if edit_distance(typo_prefix, prefix_part) <= max_distance:
            close_matches.append((word, freq))

    top_results = sorted(close_matches, key = lambda x: -x[1])[:top_n]
    return top_results

In [120]:
trie = Trie()
trie.insert("help", 8)
trie.insert("helper", 5)
trie.insert("helping", 3)
trie.insert("held", 4)

In [122]:
# Simulate user selecting a word
selected_word = "help"
trie.increase_frequency(selected_word)

Frequency of help updated to 9


True

In [124]:
trie.autocomplete("hel")

[('help', 9), ('helper', 5), ('held', 4)]

In [126]:
trie = Trie()
trie.insert("theory", 5)
trie.insert("there", 7)
trie.insert("theme", 3)

print(smart_autocomplete(trie, "hte"))

Prefix hte corrected to the
[('there', 7), ('theory', 5), ('theme', 3)]


In [128]:
print(fuzzy_autocomplete(trie, "hepl"))  # Typo of 'help'
print(fuzzy_autocomplete(trie, "hel", max_distance=0)) 

[]
[]
